## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Check for inbalanced variable
for column in application_df.columns:
    print(f"Column: {column}")
    print(application_df[column].value_counts())
    print("\n")


Column: EIN
EIN
10520599     1
626274659    1
630475330    1
630416100    1
630357662    1
            ..
383880377    1
383876652    1
383874900    1
383871370    1
996086871    1
Name: count, Length: 34299, dtype: int64


Column: NAME
NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIG

Evaluation
From the analysis above, it's clear that three specific columns—‘STATUS’, ‘SPECIAL_CONSIDERATIONS’, and ‘ASK_AMT’—stand out due to their extreme imbalance, with a single value dominating over 95% of the entries in each case. This kind of skew typically adds noise rather than useful signal, making it likely that these features did little to help the model learn meaningful patterns. In fact, their inclusion may have actively hindered performance, possibly playing a role in the model's lower accuracy metrics.

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT'], axis=1)

### Application Type

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_list = application_df.value_counts('APPLICATION_TYPE')

application_type_list

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_list[application_type_list < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

### Classification

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_list = application_df['CLASSIFICATION'].value_counts()

classification_list

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_list[classification_list < 100].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

The original classification column included around 71 distinct categories. By applying a threshold of 1000, this number was reduced to just five dominant groups, simplifying the dataset but resulting in the aggregation of many less frequent labels into a single “other” category. Notably, this "other" class became the fourth most common, suggesting that it may still contain informative patterns relevant to classification.

To address this potential loss of granularity, the threshold was lowered to 100, allowing more of the rarer categories to be preserved during preprocessing. This change aimed to enrich the dataset’s diversity and improve the model’s ability to recognize finer distinctions across a broader range of categories.



In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

application_df = pd.get_dummies(application_df, columns=categorical_columns, dtype=int)
application_df

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [9]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim = len(X.columns)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\Jitesh Makan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,209 (55.50 KB)

 Trainable params: 14,209 (55.50 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),])

In [13]:
# Train the model
nn_model_fit = nn.fit(X_train_scaled, y_train, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6937 - loss: 0.5958 - precision: 0.6894 - recall: 0.7761 - val_accuracy: 0.7223 - val_loss: 0.5653 - val_precision: 0.7257 - val_recall: 0.7668
Epoch 2/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7356 - loss: 0.5509 - precision: 0.7345 - recall: 0.7888 - val_accuracy: 0.7244 - val_loss: 0.5617 - val_precision: 0.7248 - val_recall: 0.7753
Epoch 3/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7384 - loss: 0.5435 - precision: 0.7402 - recall: 0.7955 - val_accuracy: 0.7258 - val_loss: 0.5601 - val_precision: 0.7165 - val_recall: 0.8001
Epoch 4/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7412 - loss: 0.5388 - precision: 0.7388 - recall: 0.7947 - val_accuracy: 0.7244 - val_loss: 0.5595 - val_precision: 0.7047 - val_recall: 0.8280
Epoch 5/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7348 - loss: 0.5383 - precision: 0.7300 - recall: 0.8039 - val_accuracy: 0.7279 - val_loss: 0.

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {round(model_loss * 100, 2)}%, Accuracy: {round(model_accuracy * 100, 2)}%, Precision: {round(model_precision * 100, 2)}%, Recall: {round(model_recall * 100, 2)}%")

268/268 - 0s - 982us/step - accuracy: 0.7266 - loss: 0.5499 - precision: 0.7183 - recall: 0.8016
Loss: 54.99%, Accuracy: 72.66%, Precision: 71.83%, Recall: 80.16%


In [15]:
# Export our model to HDF5 file
nn.save('exported_models/AlphabetSoupCharity_Optimized.h5')

## Re-Compile, Re-Train and Re-Evaluate the Model with extra Layer

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim = len(X.columns)))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,257 (63.50 KB)

 Trainable params: 16,257 (63.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Compile the model
nn2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),])

In [18]:
# Train the model
nn2_model_fit = nn2.fit(X_train_scaled, y_train, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6997 - loss: 0.5973 - precision_1: 0.7014 - recall_1: 0.7607 - val_accuracy: 0.7226 - val_loss: 0.5615 - val_precision_1: 0.7118 - val_recall_1: 0.8027
Epoch 2/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7392 - loss: 0.5448 - precision_1: 0.7324 - recall_1: 0.8037 - val_accuracy: 0.7178 - val_loss: 0.5615 - val_precision_1: 0.6957 - val_recall_1: 0.8327
Epoch 3/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7339 - loss: 0.5465 - precision_1: 0.7275 - recall_1: 0.8070 - val_accuracy: 0.7236 - val_loss: 0.5584 - val_precision_1: 0.7134 - val_recall_1: 0.8016
Epoch 4/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7371 - loss: 0.5422 - precision_1: 0.7319 - recall_1: 0.7997 - val_accuracy: 0.7240 - val_loss: 0.5601 - val_precision_1: 0.7111 - val_recall_1: 0.8089
Epoch 5/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7349 - loss: 0.5443 - precision_1: 0.7309 - recall_1: 0.8041 -

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = nn2.evaluate(X_test_scaled, y_test, verbose=2)
print("Extra Hidden Dense Layer:")
print(f"Loss: {round(model_loss * 100, 2)}%, Accuracy: {round(model_accuracy * 100, 2)}%, Precision: {round(model_precision * 100, 2)}%, Recall: {round(model_recall * 100, 2)}%")

268/268 - 0s - 949us/step - accuracy: 0.7272 - loss: 0.5519 - precision_1: 0.7197 - recall_1: 0.8001
Extra Hidden Dense Layer:
Loss: 55.19%, Accuracy: 72.72%, Precision: 71.97%, Recall: 80.01%


In [20]:
# Export our model to HDF5 file
nn2.save('exported_models/AlphabetSoupCharity_extraLayer_Optimized.h5')

## Re-Compile, Re-Train and Re-Evaluate the Model using tanh instead of relu

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=128, activation='tanh', input_dim = len(X.columns)))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=64, activation='tanh'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 128)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,209 (55.50 KB)

 Trainable params: 14,209 (55.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Compile the model
nn3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),])

In [23]:
# Train the model
nn3_model_fit = nn3.fit(X_train_scaled, y_train, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7043 - loss: 0.5894 - precision_2: 0.7092 - recall_2: 0.7391 - val_accuracy: 0.7219 - val_loss: 0.5661 - val_precision_2: 0.7218 - val_recall_2: 0.7749
Epoch 2/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7285 - loss: 0.5573 - precision_2: 0.7229 - recall_2: 0.7931 - val_accuracy: 0.7230 - val_loss: 0.5637 - val_precision_2: 0.7157 - val_recall_2: 0.7936
Epoch 3/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7291 - loss: 0.5545 - precision_2: 0.7236 - recall_2: 0.7885 - val_accuracy: 0.7252 - val_loss: 0.5605 - val_precision_2: 0.7194 - val_recall_2: 0.7910
Epoch 4/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7380 - loss: 0.5448 - precision_2: 0.7316 - recall_2: 0.8008 - val_accuracy: 0.7230 - val_loss: 0.5613 - val_precision_2: 0.7173 - val_recall_2: 0.7895
Epoch 5/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7300 - loss: 0.5501 - precision_2: 0.7251 - recall_2: 0.7883 -

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = nn3.evaluate(X_test_scaled, y_test, verbose=2)
print("Tanh instead of Relu:")
print(f"Loss: {round(model_loss * 100, 2)}%, Accuracy: {round(model_accuracy * 100, 2)}%, Precision: {round(model_precision * 100, 2)}%, Recall: {round(model_recall * 100, 2)}%")

268/268 - 0s - 860us/step - accuracy: 0.7276 - loss: 0.5504 - precision_2: 0.7170 - recall_2: 0.8080
Tanh instead of Relu:
Loss: 55.04%, Accuracy: 72.76%, Precision: 71.7%, Recall: 80.8%


In [25]:
# Export our model to HDF5 file
nn3.save('exported_models/AlphabetSoupCharity_tanh_Optimized.h5')

Despite the removal of three low-utility columns—‘STATUS’, ‘SPECIAL_CONSIDERATIONS’, and ‘ASK_AMT’—and the implementation of architectural variations, including the addition of a supplementary dense layer and substitution of the ReLU activation function with tanh, the model’s performance metrics remained largely unchanged. The most effective configuration yielded an accuracy of approximately 72.83% and a recall of 79.33%. This stability in performance implies that while the excluded columns introduced some degree of noise, they were not the primary limiting factors influencing model efficacy.

The persistence of these results across different configurations suggests that the principal constraint may reside within the dataset itself. It is likely that the patterns required for effective classification are either too subtle, inconsistently represented, or heavily imbalanced across classes. As such, meaningful performance gains will probably depend less on network architecture and more on advanced data preprocessing techniques, refined feature engineering, or acquisition of higher-quality input variables